In [2]:
import pandas as pd
import numpy as np
import json
import os
from pathlib import Path
import argparse
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import trange
from sklearn.metrics import balanced_accuracy_score, accuracy_score

from deeppavlov import build_model, configs

# parser = argparse.ArgumentParser()
# parser.add_argument('--files', nargs='+', help='.csv files with model answers')
# parser.add_argument('--meta', nargs='+', help='controlled params',
#                     choices=['sent', 'da'], default=['sent', 'da'])
# parser.add_argument('--out_dir', default='./analyse',
#                     help='directory to store logs')
# args = parser.parse_args()

In [14]:
class Args:
    def __init__(self):
        self.files = ["../sentiment_da_model_mean_blend.csv"]
        self.meta = ['sent', 'da']
        self.out_dir = './analyse'
args = Args()

In [3]:
def load_dataset(filename, meta1=None, meta2=None, cut_eos=True):
    df = pd.read_csv(filename, index_col=0)
    if meta1 is not None:
        df[meta1] = df['meta'].str[2].astype(int)
    if meta2 is not None:
        df[meta2] = df['meta'].str[7].astype(int)
    df.fillna('', inplace=True)
    if cut_eos:
        df.loc[:, 'gold_answer'] = df['gold_answer'].str[
                                   :-len('<|endoftext|>')]
    return df

In [4]:
da_mapping = {
    1: 'inform',
    2: 'question',
    3: 'directive',
    4: 'commissive'
}

DA_NO_HIST_CONFIG_PATH = "dialog_acts_no_hist.json"

model_no_history = build_model(DA_NO_HIST_CONFIG_PATH)

SENTIMENT_CONFIG_PATH = "sentiment_hist.json"

sent_with_hist = build_model(SENTIMENT_CONFIG_PATH)

[nltk_data] Downloading package punkt to /home/admin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/admin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/admin/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/admin/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
/home/admin/miniconda3/envs/innprac37/lib/python3.7/site-packages/cryptography/hazmat/backends/openssl/x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
2022-03-21 18:56:39.500 INFO in 'deeppavlov.models.torch_bert.torch_transformers_classifier'['torch_transformers_classifier'] at line 206: From pretrained bert-base-cased.
Some we

2022-03-21 18:56:55.877 WARNING in 'deeppavlov.core.common.registry'['registry'] at line 56: Registry name "dialog_act_hist_preprocessor" has been already registered and will be overwritten.
2022-03-21 18:56:59.433 WARNING in 'deeppavlov.core.common.registry'['registry'] at line 56: Registry name "torch_transformers_classifier" has been already registered and will be overwritten.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification

In [19]:
def apply_da_classifier_without_hist(model, df, text_col,
                                     est_col='estimated_da',
                                     step=1000):
    for i in trange(0, len(df), step):
        index = df.iloc[i:i + step, :].index
        texts = df.loc[index, text_col]
        print(f"*******************************{texts.size}************************************")
        predictions = model(list(texts))
        df.loc[index, est_col] = predictions

    df[est_col] = df[est_col].astype(np.int32)

    return df


def apply_da_classifier_with_hist(model, df, hist_col, text_col,
                                  est_col='estimated_da',
                                  step=1000, hist_sep='<|endoftext|>',
                                  num_last=100):
    for i in trange(0, len(df), step):
        index = df.iloc[i:i + step, :].index
        texts = list(df.loc[index, text_col])
        hists = [hist.split(hist_sep)[-num_last:] for hist in
                 df.loc[index, hist_col]]
        predictions = model(hists, texts)
        df.loc[index, est_col] = predictions

    df[est_col] = df[est_col].astype(np.int32)

    return df

In [6]:
sent_mapping = {
    'NEG': 1,
    'NEU': 2,
    'POS': 3
}
sent_mapping_rev = {value: key for key, value in sent_mapping.items()}

In [7]:
def apply_sent_classifier_hist(model, df, hist_col, text_col,
                               est_col='estimated_sentiment', step=1000,
                               hist_sep='<|endoftext|>', num_last=5, max_len=100):
    for i in trange(0, len(df), step):
        index = df.iloc[i:i + step, :].index
        texts = list(df.loc[index, text_col].str[:max_len])
        hists = [hist.split(hist_sep)[-num_last:] for hist in
                 df.loc[index, hist_col]]
        predictions = model(hists, texts)
        df.loc[index, est_col] = [int(p) + 2 if p != 'here.' else 2 for p in predictions]

    df[est_col] = df[est_col].astype(np.int32)

    return df


def score(df, col1, col2, stats, prefix):
    b_acc = balanced_accuracy_score(df[col1], df[col2])
    acc = accuracy_score(df[col1], df[col2])
    print(prefix)
    print(f"Balanced accuracy: {b_acc * 100:.2f}%")
    print(f"Accuracy:          {acc * 100:.2f}%")
    stats[f'{prefix}_balanced_accuracy'] = b_acc
    stats[f'{prefix}_accuracy'] = acc


def show_missmatches(f, df, gold_meta, estimated_meta,
                     gold_answer="gold_answer",
                     model_answer=None, meta_mapping=None, seed=42,
                     num_samples=10):
    bad_df = df[df[gold_meta] != df[estimated_meta]]
    np.random.seed(seed)
    samples = np.random.choice(bad_df.index, num_samples, replace=False)

    for i in samples:
        row = bad_df.loc[i, :]
        row_gold_meta = row[gold_meta]
        row_estimated_meta = row[estimated_meta]
        if meta_mapping is not None:
            row_gold_meta = meta_mapping[row_gold_meta]
            row_estimated_meta = meta_mapping[row_estimated_meta]
        print(f'gold answer:  {row[gold_answer]}', file=f)
        if model_answer is not None:
            print(f'model answer: {row[model_answer]}', file=f)
        print(f'gold da:      {row_gold_meta}', file=f)
        print(f'estimated da: {row_estimated_meta}', file=f)
        print(file=f)
        print(file=f)


def confusion_matrix(f, df, gold_meta, estimated_meta,
                     title='Confusion matrix', xlabel='Generated',
                     ylabel='Selected branch', ticklabels=None):
    values = np.unique(df[gold_meta].to_numpy())
    pairs = np.zeros((len(values), len(values)))

    for i, g_val in enumerate(values):
        window = df[df[gold_meta] == g_val]
        for j, e_val in enumerate(values):
            pairs[i, j] = len(window[window[estimated_meta] == e_val])

    pairs /= pairs.sum(axis=1)[:, None]

    plt.figure(figsize=(12, 8))
    ax = plt.axes()
    sns.heatmap(pairs, annot=True, vmin=0., vmax=1.)
    if ticklabels is not None:
        ax.set_xticklabels(list(ticklabels))
        ax.set_yticklabels(list(ticklabels))
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.savefig(f)

In [13]:
args.files

'../sentiment_da_model_mean_blend.csv'

In [15]:
for filename in args.files:
    print(f'Analysing {filename}...')
    data_name = '.'.join(Path(filename).name.split('.')[:-1])
    out_dir = Path(args.out_dir) / data_name
    os.makedirs(out_dir, exist_ok=True)

    if len(args.meta) > 1:
        df = load_dataset(filename, meta1=args.meta[0], meta2=args.meta[1])
    else:
        df = load_dataset(filename, meta1=args.meta[0])
    if 'da' in args.meta:
        df = apply_da_classifier_without_hist(model_no_history, df, 'model_answer')
    if 'sent' in args.meta:
        df = apply_sent_classifier_hist(sent_with_hist, df, 'history', 'model_answer')
    df.to_csv((out_dir / 'estimated.csv').absolute())

    stats = {}
    if 'da' in args.meta:
        score(df, 'da', 'estimated_da', stats, 'da')
        with (out_dir / 'da_errors.txt').open('w') as f:
            show_missmatches(f, df, 'da', 'estimated_da', model_answer='model_answer',
                             meta_mapping=da_mapping, num_samples=50)
        confusion_matrix((out_dir / 'da_confusion.png').absolute(), df, 'da', 'estimated_da',
                         title='Confusion matrix for generator',
                         xlabel='Generated dialog act', ticklabels=da_mapping.values())

    if 'sent' in args.meta:
        score(df, 'sent', 'estimated_sentiment', stats, 'sent')
        with (out_dir / 'sent_errors.txt').open('w') as f:
            show_missmatches(f, df, 'sent', 'estimated_sentiment',
                             model_answer='model_answer',
                             meta_mapping=sent_mapping_rev, num_samples=50)
        confusion_matrix((out_dir / 'sent_confusion.png').absolute(), df, 'sent', 'estimated_sentiment',
                         title='Confusion matrix for generator',
                         xlabel='Generated sentiment', ticklabels=sent_mapping.keys())

    with (out_dir / 'accuracy.json').open('w') as f:
        json.dump(stats, f)

Analysing ../sentiment_da_model_mean_blend.csv...


  0%|          | 0/8 [00:00<?, ?it/s]

*******************************1000************************************


ValueError: attempt to get argmax of an empty sequence

In [20]:
apply_da_classifier_without_hist(model_no_history, df, 'model_answer')

  0%|          | 0/8 [00:00<?, ?it/s]

*******************************1000************************************


ValueError: attempt to get argmax of an empty sequence

In [21]:
index = df.iloc[0:1000, :].index
texts = df.loc[index, "model_answer"]
print(f"*******************************{texts.size}************************************")
predictions = model_no_history(list(texts))
# df.loc[index, 'estimated_da'] = predictions

*******************************1000************************************


ValueError: attempt to get argmax of an empty sequence

In [22]:
index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            990, 991, 992, 993, 994, 995, 996, 997, 998, 999],
           dtype='int64', length=1000)

In [23]:
texts

0                      Yes , you can have it in return .
1      It is in Los Angeles , California . It's in th...
2      When I checked my bank account number , I got ...
3                            dollar dollars or dollars .
4      Oh . How much for the shipping charge ? I'll p...
                             ...                        
995        there is no opening around the belly button .
996    No . Don't you want to suffer the black eye of...
997                                   Horrible y sound !
998                                              Where ?
999      Surely I should have a different phone number ?
Name: model_answer, Length: 1000, dtype: object

In [25]:
model_no_history(['Yes , you can have it in return .'])

ValueError: attempt to get argmax of an empty sequence

In [26]:
[1] * 10

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]